In [1]:
import pandas as pd
import re
import datetime
import math

In [2]:
dogs = pd.read_csv("adoptions/dogs.csv", index_col=0)
dogs.head()

,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
id,,,,,,,,,,,,,,,,,,,,,
46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,Yellow / Tan / Blond / Fawn,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,NaN,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


In [3]:
# Capitalizziamo tutti i nomi in modo che siano uguali
dogs['name'] = dogs['name'].str.capitalize()

In [4]:
travels = pd.read_csv("adoptions/dogTravel.csv", index_col=0)
travels.head()

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [5]:
NST = pd.read_csv("adoptions/NST-EST2021-POP.csv", header=None, thousands='.')
NST.columns = ['state', 'population']
NST.head()

,state,population
0,Alabama,5024279
1,Alaska,733391
2,Arizona,7151502
3,Arkansas,3011524
4,California,39538223


# 1. Extract all dogs with status that is not *adoptable*

In [6]:
dogs[dogs['status'] != 'adoptable'].head()

,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
id,,,,,,,,,,,,,,,,,,,,,
41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,...,NV,89146,US,89009,2019-09-20,NaN,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,White / Cream,...,AZ,85249,US,AZ,2019-09-20,NaN,Dog,You can fill out an adoption application onlin...,80,130.77
45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,NaN,...,NY,12220,US,CT,2019-09-20,NaN,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,NaN,...,NY,12220,US,CT,2019-09-20,NaN,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,White / Cream,...,NY,12220,US,CT,2019-09-20,NaN,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61


# 2. For each (primary) breed, determine the number of dogs

In [7]:
dogs_grouped = dogs.groupby('breed_primary').count()
dogs_grouped['org_id']

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: org_id, Length: 216, dtype: int64

# 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [8]:
dogs_grouped['breed_secondary']/(dogs_grouped['org_id']-dogs_grouped['breed_secondary'])

breed_primary
Affenpinscher                        0.133333
Afghan Hound                         0.333333
Airedale Terrier                     0.900000
Akbash                               0.000000
Akita                                0.403101
                                       ...   
Wirehaired Pointing Griffon          0.000000
Wirehaired Terrier                   0.428571
Xoloitzcuintli / Mexican Hairless    0.571429
Yellow Labrador Retriever            0.645833
Yorkshire Terrier                    0.558442
Length: 216, dtype: float64

# 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [9]:
# Creiamo un subset con tutte le righe da correggere
# Troviamo queste righe cercando, nella colonna posted, celle che contengono lettere
dogsToCorrect = dogs[dogs['posted'].str.match("\D+")]

In [10]:
# creiamo una nuova lista names al cui interno metteremo i nomi corretti
names = []
for line in dogsToCorrect['name']:
    names.append(re.search('\w+', line).group().capitalize())

In [11]:
# costruiamo un subset formato solo dalle colonne sbagliate
correction = dogsToCorrect.loc[:,'status':'stateQ']
# shiftiamo il nome delle colonne del subset
# in questo modo chiamiamo le colonne dalla 24 alla 31 con i nomi dalla 25 alla 32
correction.columns = dogs.keys()[25:32]
correction.head()

,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed
id,,,,,,,
41330726,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20
38169117,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20
45833989,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20
45515547,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20
45294115,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20


In [12]:
df1 = dogsToCorrect.loc[:,:'env_cats']
df1['name'] = names
df1['status'] = None

In [13]:
df2 = dogsToCorrect.loc[:,'type.y':]

In [14]:
dogs2 = pd.concat([df1, correction, df2], axis=1)

In [15]:
# prendiamo le righe giuste sin dall'inizio cancellando con drop le righe sbagliate
dogs = dogs.drop(dogs2.index)

In [16]:
# e le aggiungiamo alle righe appena corrette
dogs = pd.concat([dogs,dogs2])

In [17]:
dogs['posted'] = pd.to_datetime(dogs['posted'], format='%Y-%m-%d %H:%M:%S')

In [18]:
dogs.groupby('breed_primary')['posted'].max()

breed_primary
Affenpinscher                       2019-09-14 10:10:51+00:00
Afghan Hound                        2019-07-27 00:38:48+00:00
Airedale Terrier                    2019-09-19 18:40:39+00:00
Akbash                              2019-08-23 17:11:04+00:00
Akita                               2019-09-20 15:19:57+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless   2019-09-08 11:15:54+00:00
Yellow Labrador Retriever           2019-09-20 06:30:27+00:00
Yorkshire Terrier                   2019-09-20 16:56:38+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

In [19]:
dogs.groupby('breed_primary')['posted'].min()

breed_primary
Affenpinscher                       2012-03-08 10:27:33+00:00
Afghan Hound                        2017-06-29 23:28:51+00:00
Airedale Terrier                    2014-06-13 12:59:36+00:00
Akbash                              2019-07-21 00:35:59+00:00
Akita                               2012-03-03 09:31:08+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2012-11-27 14:07:54+00:00
Xoloitzcuintli / Mexican Hairless   2007-02-01 00:00:00+00:00
Yellow Labrador Retriever           2010-05-31 00:00:00+00:00
Yorkshire Terrier                   2012-02-04 17:14:52+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

# 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [20]:
def sex(row):
    if row['sex'] == 'Male':
        return 1
    elif row['sex'] == 'Female':
        return -1

In [21]:
dogs['sex_imbalance'] = dogs.apply(sex, axis=1)

In [22]:
imbalance = dogs.groupby('contact_state')['sex_imbalance'].sum()

In [23]:
imbalance[imbalance == imbalance.max()]

contact_state
OH    205.0
Name: sex_imbalance, dtype: float64

In [24]:
# Oppure, per avere solo il nome
imbalance.idxmax()

'OH'

# 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [25]:
dogs.groupby(['age', 'size']).mean(numeric_only=True)[['stay_duration', 'stay_cost']]

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

# 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [26]:
dogTravel = pd.merge(dogs, travels, on='id')

In [27]:
dogTravel_group = dogTravel.groupby('id')['org_id'].count()

In [28]:
n_viaggi = dogTravel_group[dogTravel_group > 2].rename('n_viaggi')

In [29]:
pd.merge(dogs, n_viaggi, on='id')[['breed_primary', 'n_viaggi']]

,breed_primary,n_viaggi
id,,
45642530,Jindo,4
46039420,Border Collie,4
40036107,Pit Bull Terrier,3
45851842,Labrador Retriever,3
45841145,Mixed Breed,3
...,...,...
38664932,Pit Bull Terrier,3
38495992,Pit Bull Terrier,3
45588395,German Shepherd Dog,3


# 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found

In [30]:
def state(row):
    if not(pd.isnull(row['manual'])):
        return row['manual']
    else:
        return row['found']

In [31]:
travels['state'] = travels.apply(state, axis=1)

In [32]:
travels.head()

,id,contact_city,contact_state,description,found,manual,remove,still_there,state
index,,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN,Arkansas
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN,Bahamas
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN,Maryland
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN,Adaptil
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN,Afghanistan


# 9. For each state, compute the ratio between the number of travels and the population.

In [33]:
travels_NST = pd.merge(travels, NST, on='state')

In [34]:
n_viaggi2 = travels_NST.groupby('state').count()['id'].rename('n_viaggi')

In [35]:
ratio = pd.merge(n_viaggi2, NST, on='state').set_index('state')

In [36]:
ratio['ratio'] = ratio['n_viaggi']/ratio['population']

In [37]:
ratio.head()

,n_viaggi,population,ratio
state,,,
Alabama,338,5024279,0.000067
Arizona,33,7151502,0.000005
Arkansas,61,3011524,0.000020
California,279,39538223,0.000007
Colorado,11,5773714,0.000002


# 10. For each dog, compute the number of days from the posted day to the day of last access.

In [38]:
# Trasformiamo accessed in formato datetime
dogs['accessed'] = pd.to_datetime(dogs['accessed'], format='%Y-%m-%d')

In [39]:
# Prendiamo solo anno, mese e giorno da posted, in modo da eliminare le informazioni dell'ora e della time zone
posted_naive = dogs['posted'].dt.strftime('%Y-%m-%d')
posted_naive = pd.to_datetime(posted_naive, format='%Y-%m-%d')

In [40]:
dogs['accessed'] - posted_naive

id
46042150    0 days
46042002    0 days
46040898    0 days
46039877    0 days
46039306    0 days
             ...  
45916348   11 days
45733027   27 days
45413997   51 days
45406516   53 days
45264615   67 days
Length: 58180, dtype: timedelta64[ns]

In [41]:
dogs['daysPassed'] = dogs['accessed'] - posted_naive

# 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [42]:
weekPassed = dogs['daysPassed'].dt.days/7

In [43]:
dogs['weekPassed'] = weekPassed.apply(math.floor)

In [44]:
dogs.groupby('weekPassed')['org_id'].count()

weekPassed
0      9803
1      6547
2      5764
3      3353
4      2439
       ... 
729       1
746       1
811       1
812       1
852       1
Name: org_id, Length: 579, dtype: int64

# 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [46]:
duplicateCount = dogs.groupby(['breed_primary', 'sex'])['org_id'].count()

In [47]:
# Tieni solo i gruppi che contengono più di un cane
duplicates = duplicateCount[duplicateCount > 1].rename('n_cani')

In [48]:
groups = dict(list(dogs.groupby(['breed_primary', 'sex'])))

In [49]:
duplicates

breed_primary                      sex   
Affenpinscher                      Female      9
                                   Male        8
Afghan Hound                       Male        3
Airedale Terrier                   Female      8
                                   Male       11
                                            ... 
Xoloitzcuintli / Mexican Hairless  Male        7
Yellow Labrador Retriever          Female     76
                                   Male       82
Yorkshire Terrier                  Female    156
                                   Male      204
Name: n_cani, Length: 365, dtype: int64

In [118]:
# 45s
duplicati = []
for i in range(len(subset)):
        if pd.isnull(subset['desSplit'].iloc[i]).any():
            continue
        for j in range(len(subset))[i+1:]:
            match = 0
            if pd.isnull(subset['desSplit'].iloc[j]).any():
                continue
            for word in subset['desSplit'].iloc[i]:
                if word in subset['desSplit'].iloc[j]:
                    match += 1
            if match/len(subset['desSplit'].iloc[i]) >= 0.9:
                duplicati.append(subset.index[j])

In [135]:
# 9s
# implement 10% rule
duplicati = []
# loop for the first row
for i in range(len(subset)):
    # If the text is "nan" or the row has already been catalogated as a duplicate, skip to the next row
    if pd.isnull(subset['desSplit'].iloc[i]).any() or subset.index[i] in duplicati:
        continue
    # loop for the second row
    for j in range(len(subset))[i+1:]:
        Nmatch = 0
        # set "tenP" as the number that indicates 10% of the words
        tenP = len(subset['desSplit'].iloc[i])/10
        # If the text is "nan" or the row has already been catalogated as a duplicate, skip to the next row
        if pd.isnull(subset['desSplit'].iloc[j]).any() or subset.index[j] in duplicati:
            continue
        # Loop through the words of the first row
        for word in subset['desSplit'].iloc[i]:
            # If the word of the first row is not in the second row, then add 1 to Nmatch
            if word not in subset['desSplit'].iloc[j]:
                Nmatch += 1
            # When Nmatch reaches the 10% line, we know that for sure less than 90% of the words are equal, so we skip to the next iteration
            if Nmatch > tenP:
                break
        # if the "break" statement never occured, than the 10% line has never been reached, so more than 90% of the words are the same
        # so we catalog the second row as "duplicate"
        else:
            duplicati.append(subset.index[j])

In [51]:
def split(row):
    if not(pd.isnull(row['description'])):
        return re.sub(r'[^\w\s]', '', row['description'].lower()).split(" ")
    else:
        return [math.nan]

In [54]:
%%time
duplicati = []
num = 0
# for every group that we found through the previous groupby
for key in duplicates.keys():
    print(str(num) + "/ 365")
    num += 1
    subset = groups[key]
    desSplit = subset.apply(split, axis=1).rename('desSplit')
    subset = pd.merge(subset, desSplit, on = 'id')
    print('merging done')
    # loop for the first row
    for i in range(len(subset)):
        # If the text is "nan" or the row has already been catalogated as a duplicate, skip to the next row
        if pd.isnull(subset['desSplit'].iloc[i]).any() or subset.index[i] in duplicati:
            continue
        # loop for the second row
        for j in range(len(subset))[i+1:]:
            Nmatch = 0
            # If the text is "nan" or the row has already been catalogated as a duplicate, skip to the next row
            if pd.isnull(subset['desSplit'].iloc[j]).any() or subset.index[j] in duplicati:
                continue
            if len(set(subset['desSplit'].iloc[i]) - set(subset['desSplit'].iloc[j]))/len(subset['desSplit'].iloc[i]) <= 0.1:
                duplicati.append(subset.index[j])

0/ 365
merging done
1/ 365
merging done
2/ 365
merging done
3/ 365
merging done
4/ 365
merging done
5/ 365
merging done
6/ 365
merging done
7/ 365
merging done
8/ 365
merging done
9/ 365
merging done
10/ 365
merging done
11/ 365
merging done
12/ 365
merging done
13/ 365
merging done
14/ 365
merging done
15/ 365
merging done
16/ 365
merging done
17/ 365
merging done
18/ 365
merging done


KeyboardInterrupt: 

In [110]:
len(duplicati)

4461

In [112]:
dogs = dogs.drop(duplicati)

In [113]:
len(dogs)

53719

In [116]:
duplicateCount = dogs.groupby(['breed_primary', 'age', 'sex', 'size', 'name','contact_city'])['org_id'].count()

In [117]:
duplicates = groups[groups>1]

In [118]:
groups = dict(list(dogs.groupby(['breed_primary', 'age', 'sex', 'size', 'name','contact_city'])))

In [120]:
duplicati = []
for i in duplicates.keys():
    for j in groups[i].index[1:]:
        duplicati.append(j)

In [121]:
len(duplicati)

152

In [122]:
dogs = dogs.drop(duplicati)

KeyError: '[45871731, 45842477, 46023963, 46019974, 46019762, 45989321, 45872595, 43853984, 43170920, 42668575, 42087185, 45639446, 45252099, 44517341, 46026337, 45831359, 45384056, 45986905, 45925981, 45925980, 45925975, 45925979, 45925974, 45925982, 45925976, 45555559, 45517678, 44399248, 45955061, 45300339, 44967207, 44434059, 45717188, 46028452, 46028451, 45949697, 45862141, 45862119, 45862106, 45861529, 45031624, 44978859, 44970891, 45677559, 46030501, 45931729, 45560548, 45685284, 45790697, 45985650, 44968255, 44837270, 44790314, 44742449, 44545963, 45732409, 46021152, 46020135, 43853983, 36745161, 45983609, 45782390, 45383992, 44425818, 45379615, 45092336, 44041085, 45861413, 44969459, 45988578, 44564534, 46006005, 44088225, 44009169, 40954349, 39638646, 45409939, 44631651, 44627406, 44417986, 45356571, 46020644, 45695457, 45599092, 45288944, 45138812, 45097240, 45029817, 45968623, 45677671, 45677650, 45335660, 44942074, 45764629, 45764537, 45758174, 45702107, 45695888, 45310401